<a href="https://colab.research.google.com/github/GitHub-Harshita/MachineLearningAlgorithms/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PyGuru.pdf to PyGuru.pdf


In [ ]:
import numpy as np
import faiss
import torch
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ----------------------------
# 1. Load Embedding Model
# ----------------------------
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# ----------------------------
# 2. Load Free LLM
# ----------------------------
model_name = "google/flan-t5-base"   # If slow, change to flan-t5-small
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ----------------------------
# 3. Read PDF
# ----------------------------
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text
    return text

# ----------------------------
# 4. Chunk Text (Improved with overlap)
# ----------------------------
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

# ----------------------------
# 5. Create FAISS Index (Cosine similarity)
# ----------------------------
def create_faiss_index(chunks):
    embeddings = embed_model.encode(chunks, normalize_embeddings=True)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(np.array(embeddings))
    return index

# ----------------------------
# 6. Retrieve Relevant Chunks
# ----------------------------
def retrieve(query, chunks, index, top_k=5):
    query_embedding = embed_model.encode([query], normalize_embeddings=True)
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [chunks[i] for i in indices[0]]

# ----------------------------
# 7. Generate Answer
# ----------------------------
def generate_answer(query, context_chunks):
    context = "\n\n".join(context_chunks)

    prompt = f"""
    You are a document question-answering assistant.
    Answer ONLY from the context below.
    If answer not found, say: "Answer not found in document."

    Context:
    {context}

    Question:
    {query}
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    outputs = llm.generate(**inputs, max_new_tokens=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# ----------------------------
# 8. Main Execution
# ----------------------------
pdf_file = list(uploaded.keys())[0]  # Automatically get uploaded filename

text = load_pdf(pdf_file)
chunks = chunk_text(text)
index = create_faiss_index(chunks)

print("RAG system ready! Ask questions below.\n")

while True:
    query = input("Ask a question (type 'exit' to stop): ")
    if query.lower() == "exit":
        break
    retrieved_chunks = retrieve(query, chunks, index)
    answer = generate_answer(query, retrieved_chunks)
    print("\nAnswer:", answer)
    print("\n" + "-"*60 + "\n")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


RAG system ready! Ask questions below.

Ask a question (type 'exit' to stop): what is pyguru?

Answer: an adaptive, intelligent AI tutor

------------------------------------------------------------

Ask a question (type 'exit' to stop): what is goal of pyguru?

Answer: make Python learning interactive, personalized, and effective

------------------------------------------------------------

Ask a question (type 'exit' to stop): exit
